## Discovering impact of the Series 'Euphoria' through NLP
### Analysis based on posts and comments on the `r/euphoria` subreddit

#### Question & Data Cleaning

Analysis plan:  

0. identify question(s) and data sources
1. clean data to get it into a standard format for further analysis
   1. corpus (collection of texts) - to dataframe using `pandas`
   2. document-term matrix - clean, tokenize, tdm
2. EDA
3. topic modeling based on comments referencing 'rue' posts on r/euphoria
4. *network analysis between users of drug and euphoria communities* - time permitting
5. aspect based sentiment analysis of euphoria drug-related comments
6. *topic modeling based on comments that refer to euphoria on r/opioids, r/cannabis, r/benzodiazepenes* - time permitting

#### 0. What is our question?

*What about the drug portrayal on HBO's Euphoria makes it engaging to fans?*
*Specifically, what topics emerge in the commments and is there a observable difference between seasons 1 and 2?*

I: the question, "what makes viewers of HBO's Euphoria engaged in online discourse on Reddit - Seasons 1 and 2?"  
O: data that cleaned, organized, in a standard format that can be used in future analysis  

**Data Source(s)**  
1. Reddit - `r/euphoria`
   1. posts/comments
   2. filtering comments on date and 'Rue'
      1. S1:
      2. S2: jan 9 - feb 7, 2022

**Limit Scope**
- Using library `psaw`, pull posts that mention 'Rue' during the time frame
- Using post ids, pull all comment trees for each post
- Experiment with tree depth : 100%, 85%, 50% or top N

**Data Gathering**
- `psaw`
  - wrapper around pushift.io reddit api
- `pickle`
  - saving data for later
- `pandas`
  - exporting data to csv


In [1]:
# load libraries
import pandas as pd
import numpy as np
import pickle

In [4]:
# load data
from pandas import read_pickle

raw = read_pickle('../dat/s2_rue_comments.pkl')

In [5]:

comments = raw[raw[0] != '[deleted]']
# remove any row that contatins an automatically generated response by the bot
pattern_del = 'Thank you for your submission'
filtered = comments[0].str.contains(pattern_del, na = False)
comments = comments[~filtered]
# remove solicitation comments
pattern_ads = 'Paypal'
filtered_ads = comments[0].str.contains(pattern_ads, na = False)
comments = comments[~filtered_ads]
# more solicitations
pattern3 = 'Here is [Rue 21 Coupon Code]'
comments = comments[~comments[0].str.contains(pattern3, na = False)]


In [6]:
# pattern4 = '*PAYPAL INVOICE ONLY*'
# comments = comments[~comments[0].str.contains(pattern4, na = False)]
# 
pattern5 = 'JAX\_Jacksonville\_127'
comments = comments[~comments[0].str.contains(pattern5, na = False)]
# some commentary on edgar allen poe?
pattern7 = 'The word detective did not'
comments = comments[~comments[0].str.contains(pattern7, na = False)]
# 
pattern8 = 'On my way to trinoma/vertis north'
comments = comments[~comments[0].str.contains(pattern8, na = False)]
# 
pattern10 = 'Shipping:'
comments = comments[~comments[0].str.contains(pattern10, na = False)] 
# promoting music
pattern11 = 'Albums'
comments = comments[~comments[0].str.contains(pattern11, na = False)]


In [7]:
# remove non=english comments
from langdetect import detect

# add a new column for language
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

comments['lang'] = comments[0].apply(detect_language)


In [8]:
# filter for english
comments = comments[comments['lang'] == 'en']

In [9]:
comments_ls = comments[0].tolist()

#### 1. Clean data  

Common steps:
- remove punctuation  
- lowercase letters  
- remove numbers

Future steps after tokenization:  
- stem
- lemmatize
- combine phrases like 'thank you' to bigrams

In [10]:
# load libraries
from string import punctuation
import re
import unicodedata

In [8]:
# clean and remove numbers
# def cleaning_numbers(data):
#     return re.sub('[0-9]+', '', data)

In [11]:
# using regex - clean and remove URLs
def cleaning_URLs(text):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',text)

In [12]:
def clean_round1(text):
    # convert all to string
    text = str(text)
    # lower
    text = text.lower()
    # text in squre brackets
    text = re.sub('\[.*?\]', ' ', text)
    # urls
    text = cleaning_URLs(text)
    # punctuation
    text = re.sub('[%s]' % re.escape(punctuation), ' ', text)
    # remove numbers
    text = re.sub('[0-9]+', '', text)
    # it looks like there are edits being made to comments
    # remove any instance of edit:
    text = re.sub('edit:', '', text)
    # remove any user handles
    text = re.sub('@[a-zA-Z0-9_]+', '', text)
    return text

round1 = lambda x: clean_round1(x)

In [13]:
data_clean = []
for comment in comments_ls:
    data_clean.append(round1(comment))

In [14]:
# round two of cleaning: remove line breaks, emojis, qutes, etc.
def clean_round2(text):
    text = re.sub('\n', '', text)
    # text = re.sub('[''""...]', '', text)
    # remove emojis
    text = ''.join(c for c in text if not unicodedata.combining(c))
    return text

round2 = lambda x: clean_round2(x)

In [15]:
data_clean2 = []
for comment in data_clean:
    data_clean2.append(round2(comment))

In [16]:
# round 3 cleaning: expand contractions: i'd, you've, you're etc
import contractions
def clean_round3(text):
    # expand contractions
    text = contractions.fix(text)
    return text

round3 = lambda x: clean_round3(x)

In [17]:
data_clean3 = []
for comment in data_clean2:
    data_clean3.append(round3(comment))

*because of the cleanning of repeated characters affecting words with double letters (addict, struggle), do stemming and lemmatizing first, then clean repeated chars*

In [18]:
# another round with lemmatization and stemming
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

def clean_round3a(text):
    # stemming
    # stemmer = PorterStemmer()
    # text = ' '.join([stemmer.stem(word) for word in text.split()])
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    return text

round3a = lambda x: clean_round3a(x)


In [19]:
data_clean3a = []
for comment in data_clean3:
    data_clean3a.append(round3a(comment))

In [67]:
# grab the 500 most common words in all the comments to use to populate a
# list of words to be excluded in the modify function
# def get_top_n_words(corpus, n=None):
#     vectorizer = CountVectorizer(max_features=n, stop_words='english')
#     vectorizer.fit_transform(corpus)
#     bag_of_words = vectorizer.vocabulary_
#     # sort the words by their frequency
#     bag_of_words = sorted(bag_of_words.items(), key=lambda x: x[1], reverse=True)
#     return bag_of_words

# top = get_top_n_words(data_clean3a, n=500)

In [20]:
# remove accented characters
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

# clean and remove repeated characters
# def cleaning_repeating_char(text):
#     return re.sub(r'(.)1+', r'1', text)

In [21]:
do_not_mod_ls = ['good', 'spelling', 'telling', 'addiction','addicts', 'finally', 'all', 'personally', 'struggles', 'really', 'cassie', 'free', 
                 'feeling', 'elliot', 'maddy', 'literally', 'getting', 'better', 'actually', 'totally', 'telling', 'supposed', 'teen',
                 'stuff', 'sorry', 'soon', 'sells', 'sees', 'schools', 'rooms', 'redeeming', 'reddit', 'putting', 'pretty', 'official', 'need',
                 'messages', 'matters', 'looks', 'little', 'kills', 'issues', 'horror', 'hell', 'happens', 'happy', 'gonna', 'getting', 'especially',
                 'different', 'classic', 'businesses', 'attention', 'basically', 'apples', 'weeks', 'streets', 'needles', 'planning', 'full', 'fully',
                 'agreed', 'will', 'been', 'seeds', 'desserts', 'google', 'seen', 'addictions', 'foot', 'funny', 'comments', 'comment', 'poll',
                 'channel', 'well', 'wall', 'hall']

from operator import contains
def modify(s):
    # split comments into words
    comment = []
    for word in s.split():
        if any(word in x for x in do_not_mod_ls if contains(x, word)):
            comment.append(word)
        else:
            word = re.sub(r'([a-z])\1+', r'\1', word)
            comment.append(word)
    # join the words back together
    comment = ' '.join(comment)
    return comment
    
# print(modify('good'))
# print(modify('waaayyy'))

In [55]:
# modify('switching to needle is baaaaaddddddd she manipulated her into needle use planning to pimp her out to pay her debt dark af')

'switching to needle is bad she manipulated her into needle use planning to pimp her out to pay her debt dark af'

In [22]:
# remove repeating characters and non unicode characters
def clean_round4(text):
    # text = remove_accented_chars(text)
    # text = cleaning_repeating_char(text)
    # remove non unicode characters
    text = re.sub('[^\x00-\x7F]+', '', text)
    # remove repeating characters
    text = modify(text)
    text = text.strip()
    return text

round4 = lambda x: clean_round4(x)

In [23]:
data_clean4 = []
for comment in data_clean3a:
    data_clean4.append(round4(comment))

there are a lot of words with similar roots: play, played, playing

also, somehow a lot of random posts ended up in the clean set:  
- ' shiping usa only payment paypal fampf buyer pay fe for paypal gampsdecants ml or ml htpsimgurcomalhnzx htpsimgurcomaiozvyq of botles and respective level measured with a syringe for reference ptfe tape betwen botle and nozle thread and parafilm around where nozle and botle met decants are individualy buble wraped before going inside zip bag and then inside buble mailersthanks for loking please chatpm if interested or if you have any questionsampxb htpsimgurcomalhnzx link for a photo of botlesmlmlgardenia antiguaorangerie venisepivoine suzhourose darabierogue malachitesable nuitvert malachiteampxb htpsimgurcomasvqrut link for a photo of botlesmlml rue cambonbeigebois de ilescoromandelcristale edtcristale eau vertela pausale liono edtno edpno poudreno sycomoreampxb htpsimgurcomausue link for a photo of botlesmlmlambre nuitbois dargentcuir canagebalade sauvagebele de jourdioramoureau noirefeve delicieusegrand balgris diorholy peonyjasmin de angesla cole noiremilylaforetmitzahnew lok oud ispahanoud rosewodpatchouli imperialpurple oudsantal noirvanila dioramavetiverampxb htpsimgurcomagoari link for a photo of botlesmlmlvelvet amber sunvelvet desert oudvelvet exotic leathervelvet incensovelvet mimosa blomvelvet tender oudampxb htpsimgurcomaksojbo link for a photo of botlesmlmlmlcarnal flowerlys mediteranethe monthe nightampxb htpsimgurcomazdbkwra link for a photo of botlesmlmla chant for the nympha song for the rosethe eye of the tigerthe voice of the snakeampxb htpsimgurcomahipx link for a photo of botlesmlmlangelique noirebois darmeniecuir belugaencens mythiquejoyeuse tubereuseneroli outrenoirsantal royalshalimar milsime vanila planifoliampxb htpsimgurcomalibgsh link for a photo of botlesmlmlamyris homeaqua universalis fortegrand soiroud extraitoud satin modampxb htpsimgurcomaeysuwsh link for a photo of botlesmlmlbabylondark lightday for nightdesert serenademarienbadmiracle of the roseampxb htpsimgurcomagdqne link for a photo of botlesmlmlambre sultanborneo cuir mauresquede profundisfumerie turquetubereuse crimineleampxb htpsimgurcomajxlzlf link for a photo of botlesmlml rue de belechase saint place sulpiceatlas gardencabancaftancapelinexquisite embroiderymagnificent goldsaharienesplendid wodtrenchtuxedoveloursvinyle'

**Organize data**  

1. Corpus = `data_clean4`
2. TDM

In [27]:
data_clean4_df = pd.DataFrame(data_clean4)

In [28]:
exclude = [
    18991	,
18062	,
18063	,
18063	,
18271	,
18271	,
18300	,
18300	,
18337	,
18337	,
18474	,
18474	,
18621	,
18621	,
18622	,
18622	,
18629	,
18629	,
18639	,
18639	,
18666	,
18666	,
18667	,
18667	,
18668	,
18668	,
18686	,
18686	,
18690	,
18690	,
18695	,
18695	,
18697	,
18697	,
18742	,
18742	,
18753	,
18753	,
18845	,
18846	,
18846	,
18848	,
18848	,
18851	,
18870	,
18870	,
18871	,
18871	,
18872	,
18872	,
18873	,
18873	,
18874	,
18874	,
18875	,
18875	,
18876	,
18876	,
18880	,
18880	,
18881	,
18881	,
18882	,
18882	,
18884	,
18884	,
18885	,
18885	,
18886	,
18886	,
18887	,
18887	,
18888	,
18888	,
18890	,
18890	,
18891	,
18891	,
18895	,
18895	,
18896	,
18896	,
18897	,
18897	,
18899	,
18899	,
18900	,
18900	,
18902	,
18902	,
18903	,
18903	,
18904	,
18904	,
18931	,
18931	,
18949	,
18949	,
18950	,
18950	,
18951	,
18951	,
18952	,
18952	,
18953	,
18953	,
18954	,
18954	,
18955	,
18955	,
18956	,
18956	,
18957	,
18957	,
18958	,
18958	,
18962	,
18962	,
18980	,
18980	,
18981	,
18981	,
19003	,
19003	,
19009	,
19009	,
19015	,
19015	,
19016	,
19016	,
19018	,
19018	,
19020	,
19020	,
19021	,
19021	,
19023	,
19023	,
19024	,
19024	,
19027	,
19027	,
19028	,
19028	,
19029	,
19029	,
19093	,
19093	,
19094	,
19094	,
19095	,
19095	,
19096	,
19096	,
19098	,
19098	,
19099	,
19099	,
19100	,
19100	,
19101	,
19101	,
19102	,
19102	,
19104	,
19104	,
19146	,
19146	,
19249	,
19249
]

In [29]:
# distinct
exclude = list(set(exclude))

In [30]:
# remove excluded indices from data_clean4_df
data_clean5_df = data_clean4_df.drop(data_clean4_df.index[exclude])

In [31]:
# remove any empty rows
data_clean5_df = data_clean5_df[data_clean5_df[0] != '']
data_clean5_df = data_clean5_df[data_clean5_df[0] != ' ']

In [32]:
# pickle it
data_clean5_df.to_pickle('../dat/corpus.pkl')

In [48]:
# from nltk.stem.snowball import SnowballStemmer
 
# # experiment with stemming
# data_clean5_df_a = data_clean5_df[0].apply(lambda x: ' '.join([SnowballStemmer(language = 'english').stem(word) for word in x.split()]))

In [33]:
# create document-term matrix
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean5_df[0])
data_tdm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_tdm_index = data_tdm.index
data_tdm

,ab,aback,abaedefabdfef,abafbfbedbada,abandon,abandoned,abandoning,abandonment,abashed,abc,...,zora,zorbcaps,zoted,zouabi,zoya,zqcsrpwsge,zrue,zs,zsuzsana,zy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19161,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19162,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19163,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19164,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


meh, not sure how i feel about stemming and lemmatizing.
creates weird words that i think are more noisy than helpful.

In [34]:
# pickle for later use
data_tdm.to_pickle('../dat/tdm.pkl')